In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time 

from qcodes import Instrument
from qcodes import new_experiment
from qcodes.dataset.plotting import plot_by_id
from qcodes.instrument.parameter import ManualParameter
from qcodes.sweep import sweep, getter, run

In [5]:
class SomeInstrument(Instrument): 
    def __init__(self, name, time_constant): 
        super().__init__(name)
        
        self._time_constant = time_constant
        self._previous_value_time = time.time()
        self._previous_value = 0
        self._current_value = 0
        self._set_point = 0
        
    def _update(self): 
        self._previous_value = self._current_value
        self._previous_value_time = time.time()
        
    def _x_set(self, set_point): 
        self._set_point = set_point
        self._update()
    
    def _x_get(self): 
        dt = time.time() - self._previous_value_time
        fac = np.exp(-dt/self._time_constant)
        self._current_value = self._previous_value * (1 - fac) + self._set_point * fac
        
        self._update()
        
        return self._current_value

In [10]:
x = ManualParameter("x", "V")
x.set(0)

m = ManualParameter("m", unit="A")
m.get = lambda: x() + np.random.normal(0, 0.1)

In [12]:
# First make a time trace where we monitor m over a 3 second period with a 0.1 second interval 
sweep_object = TimeTrace(0.1, 3)(m) 

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

time [s] | m [A]
 0.0	 0.00894167827849132
 0.10648560523986816	 0.06670084438414951
 0.20650124549865723	 0.06639825592671711
 0.31625843048095703	 -0.10587048751525865
 0.42511796951293945	 -0.006136029153138839
 0.5271973609924316	 -0.06045104528796215
 0.636444091796875	 -0.014141384006547865
 0.7432172298431396	 -0.17902305483698197
 0.8443684577941895	 0.007645211159260964
 0.9449670314788818	 -0.11843379652704354
 1.0453166961669922	 -0.14544003877061282
 1.1526365280151367	 0.11230478735571935
 1.2528152465820312	 -0.002040498032416193
 1.3571374416351318	 0.12674765767334667
 1.457146406173706	 0.0041058545663942
 1.5575840473175049	 -0.08073127423303078
 1.657594919204712	 -0.04451702787389078
 1.7576162815093994	 0.1312532919084013
 1.862701416015625	 0.15122935131349846
 1.962709903717041	 -0.09438623482527364
 2.0717520713806152	 -0.11192636439574416
 2.172987699508667	 -0.10120819515969436
 2.273402690887451	 0.06039268894502599
 2.3749208450317383	 -0.013525674399467797


In [15]:
# We can add another axis to the time trace
sweep_object = sweep(x, [0, 1, 2, 3, 4])(
    TimeTrace(0.1, 3)(m) 
)

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

x []	time [s] | m [A]
 0	 0.0	 -0.044704896037345926
 0	 0.10100483894348145	 -0.052714593045545814
 0	 0.2036604881286621	 -0.03517505276732275
 0	 0.30377769470214844	 0.04708196485144053
 0	 0.4050900936126709	 -0.13675631421304596
 0	 0.5054759979248047	 -0.015037487777234675
 0	 0.6054918766021729	 0.034477022117996255
 0	 0.7079353332519531	 0.02796730445733341
 0	 0.8105437755584717	 0.0537174368471281
 0	 0.9111924171447754	 0.1739870996550772
 0	 1.0141839981079102	 0.1147032029797714
 0	 1.1142430305480957	 0.13387434749702248
 0	 1.2161996364593506	 -0.016380447180094883
 0	 1.3168959617614746	 -0.11363121582173791
 0	 1.417896032333374	 0.08696627057062999
 0	 1.5189008712768555	 0.0724406818137297
 0	 1.6189043521881104	 0.028781869170551984
 0	 1.7199130058288574	 -0.1624603424934486
 0	 1.8297784328460693	 0.055632464762271516
 0	 1.9298217296600342	 0.12181439634455622
 0	 2.0345377922058105	 0.13972971724946925
 0	 2.1436400413513184	 0.025089216134990114
 0	 2.2479565

In [27]:
# Finally, we can say to sweep x over setpoints, measure m at each set point and at each set point give the time. Since now
# the total time of the time trace depends on the parameter sweep, we put None for the thrid argument (which means measure
# for an infinite time)
sweep_object = time_trace(sweep(x, [0, 1, 2, 3, 4])(m), 0.3, None)

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

x []	time [s] | m [A]
 0	 0.0	 0.15895637424312647
 1	 0.302570104598999	 1.130308903350619
 2	 0.6065046787261963	 2.008630758448491
 3	 0.9105892181396484	 3.1579997522932994
 4	 1.219590187072754	 4.062669917277389


In [44]:
class A: 
    def __init__(self): 
        self._a = {}
    
    @property
    def a(self): 
        return self._a

In [45]:
aa = A()

In [46]:
aa.a["a"] = 1

In [47]:
aa.a

{'a': 1}